# Capstone: Portfolio Optimization

In [1]:
#Base
import numpy as np
import pandas as pd
from datetime import datetime

#Visualization
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

#Data Optimization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

#Analysis
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import cvxpy as cp


# 1. Data Wrangling

### 1.1 Upload the csv files

In [2]:
df=pd.read_csv('forecast2.csv')

### 1.2 Data Exploration & Preparation

In [3]:
#We can see all the columns with the close suffix that is the closing price
#and volume that is the number of transactions.
df.head(1)

,date,AAL_close,AAPL_close,AAP_close,ABC_close,ABT_close,ACN_close,ADBE_close,ADI_close,ADM_close,...,WU_close,WYNN_close,WY_close,XEL_close,XOM_close,XRAY_close,XRX_close,YUM_close,ZBH_close,ZION_close
0,2017-02-07,45.169998,32.8825,159.179993,90.330002,42.889999,115.919998,114.959999,76.309998,44.59,...,19.84,97.010002,32.049999,41.66,82.769997,57.43,28.280001,66.489998,114.669907,42.82


In [4]:
df = df.iloc[:, 1:477].copy()
df

,AAL_close,AAPL_close,AAP_close,ABC_close,ABT_close,ACN_close,ADBE_close,ADI_close,ADM_close,ADP_close,...,WU_close,WYNN_close,WY_close,XEL_close,XOM_close,XRAY_close,XRX_close,YUM_close,ZBH_close,ZION_close
0,45.169998,32.882500,159.179993,90.330002,42.889999,115.919998,114.959999,76.309998,44.590000,96.290001,...,19.840000,97.010002,32.049999,41.660000,82.769997,57.430000,28.280001,66.489998,114.669907,42.820000
1,45.060001,33.009998,162.729996,91.070000,42.400002,115.430000,116.129997,77.480003,43.459999,96.580002,...,19.910000,97.180000,32.410000,42.060001,81.480003,57.320000,29.440001,66.589996,113.737862,42.169998
2,46.299999,33.105000,164.830002,91.650002,42.540001,116.980003,116.440002,76.510002,43.540001,97.650002,...,20.379999,98.879997,33.169998,41.759998,81.839996,58.610001,29.040001,67.389999,114.174759,43.150002
3,46.450001,33.029999,164.050003,91.169998,42.740002,117.599998,116.849998,76.529999,43.830002,97.820000,...,19.740000,98.900002,33.160000,41.939999,82.519997,58.389999,29.200001,68.000000,113.912621,43.570000
4,46.450001,33.029999,164.050003,91.169998,42.740002,117.599998,116.849998,76.529999,43.830002,97.820000,...,19.740000,98.900002,33.160000,41.939999,82.519997,58.389999,29.200001,68.000000,113.912621,43.570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,28.969632,57.992860,98.719618,58.487037,71.907837,197.297586,356.964530,119.292177,37.066706,147.932503,...,18.841888,161.131250,29.865512,54.928797,72.446008,18.136782,19.757003,115.252506,102.203631,66.076984
1092,28.847834,57.730023,101.336671,58.171290,72.129965,194.427343,356.839530,118.557094,37.817457,146.560032,...,18.717348,157.131250,29.458970,55.212085,73.226655,18.060567,19.605733,114.869229,101.678442,65.084797
1093,28.789417,57.781070,100.861992,57.391559,72.446162,194.222171,358.152030,118.188459,39.535250,146.440144,...,18.482817,158.131250,29.192020,55.308835,73.137487,17.564447,19.712823,114.766718,101.656250,64.623859
1094,28.498552,57.298763,105.168514,58.124145,71.747631,192.615958,352.527030,116.992013,39.871287,145.074275,...,18.615699,148.131250,28.859703,55.513570,72.788875,18.283901,19.179186,114.164705,101.730038,63.748859


# 2. Analysis of correlations

In [5]:
correlation_matrix = df.corr()

# Find the 30 stocks with the minimal correlation
min_correlation_stocks = correlation_matrix.min().nsmallest(30).index

# Subset the original DataFrame with the selected stocks
selected_stocks = df[min_correlation_stocks]

In [6]:
expected_returns = df.mean()

In [7]:
weights = cp.Variable(len(df.columns))

In [8]:
expected_returns = df.mean()
covariance_matrix = df.cov()

In [9]:
# Calculate the daily returns for each stock
returns_df = df.pct_change().dropna()
returns_df

,AAL_close,AAPL_close,AAP_close,ABC_close,ABT_close,ACN_close,ADBE_close,ADI_close,ADM_close,ADP_close,...,WU_close,WYNN_close,WY_close,XEL_close,XOM_close,XRAY_close,XRX_close,YUM_close,ZBH_close,ZION_close
1,-0.002435,0.003877,0.022302,0.008192,-0.011425,-0.004227,0.010177,0.015332,-0.025342,0.003012,...,0.003528,0.001752,0.011232,0.009602,-0.015585,-0.001915,0.041018,0.001504,-0.008128,-0.015180
2,0.027519,0.002878,0.012905,0.006369,0.003302,0.013428,0.002669,-0.012519,0.001841,0.011079,...,0.023606,0.017493,0.023450,-0.007133,0.004418,0.022505,-0.013587,0.012014,0.003841,0.023239
3,0.003240,-0.002266,-0.004732,-0.005237,0.004701,0.005300,0.003521,0.000261,0.006661,0.001741,...,-0.031403,0.000202,-0.000301,0.004310,0.008309,-0.003754,0.005510,0.009052,-0.002296,0.009733
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,-0.004032,-0.002771,0.011423,0.010298,-0.001250,-0.000120,-0.002794,-0.001780,0.001528,-0.001149,...,0.003627,-0.018278,-0.000744,-0.000649,-0.003904,0.013311,-0.005604,-0.001704,-0.000408,-0.003300
1092,-0.004204,-0.004532,0.026510,-0.005399,0.003089,-0.014548,-0.000350,-0.006162,0.020254,-0.009278,...,-0.006610,-0.024824,-0.013612,0.005157,0.010776,-0.004202,-0.007657,-0.003326,-0.005139,-0.015016
1093,-0.002025,0.000884,-0.004684,-0.013404,0.004384,-0.001055,0.003678,-0.003109,0.045423,-0.000818,...,-0.012530,0.006364,-0.009062,0.001752,-0.001218,-0.027470,0.005462,-0.000892,-0.000218,-0.007082
1094,-0.010103,-0.008347,0.042697,0.012765,-0.009642,-0.008270,-0.015706,-0.010123,0.008500,-0.009327,...,0.007189,-0.063239,-0.011384,0.003702,-0.004767,0.040961,-0.027071,-0.005246,0.000726,-0.013540


In [10]:
# Calculate the daily returns for each stock
returns_df = df.pct_change().dropna()

# Calculate the expected returns and covariance matrix
expected_returns = returns_df.mean()
covariance_matrix = returns_df.cov()

# Define the objective function to maximize the Sharpe ratio
def objective_function(weights):
    returns = np.dot(weights, expected_returns)
    volatility = np.sqrt(np.dot(weights, np.dot(covariance_matrix, weights.T)))
    sharpe_ratio = returns / volatility
    return -sharpe_ratio  # Negate for maximization

# Define the constraints (e.g., weights sum up to 1)
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights equal to 1
               {'type': 'ineq', 'fun': lambda x: x}]  # Non-negative weights

# Define the bounds for the weights (0 <= weight <= 1)
bounds = [(0, 1)] * len(df.columns)

# Set an initial guess for the weights
initial_guess = [1 / len(df.columns)] * len(df.columns)

# Solve the optimization problem
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = result.x

# Create a DataFrame with the optimal portfolio weights
portfolio_df = pd.DataFrame({'Stock': df.columns, 'Weight': optimal_weights})

# Sort the DataFrame by weights in descending order
portfolio_df = portfolio_df.sort_values('Weight', ascending=False)

# 3. Stock Selection

In [11]:
# Normalize the weights of the top 30 stocks so that they sum up to 1
selected_stocks = portfolio_df[:30]
normalized_weights = selected_stocks['Weight'] / np.sum(selected_stocks['Weight'])

# Assign the normalized weights back to the selected_weights DataFrame
selected_stocks['Normalized_Weight'] = normalized_weights

selected_stocks = selected_stocks.drop(columns=['Weight'])

selected_stocks 

/var/folders/l_/s2stp_4940d5h7pgt3_4rl040000gn/T/ipykernel_4023/995650356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_stocks['Normalized_Weight'] = normalized_weights


,Stock,Normalized_Weight
30,AMT_close,0.116088
78,CME_close,0.111576
258,NEE_close,0.110480
259,NEM_close,0.091836
18,AJG_close,0.078162
306,REGN_close,0.050891
337,STZ_close,0.045503
389,XEL_close,0.043567
195,ISRG_close,0.042294
69,CHD_close,0.037976


In [12]:
selected_stocks.to_csv('selected_stocks(forecast)2.csv',index=False)

# 4. Repeating the process for historical data

In [13]:
df=pd.read_csv('historical2.csv')

In [14]:
df = df.iloc[:, 1:477].copy()

In [15]:
correlation_matrix = df.corr()

# Find the 30 stocks with the minimal correlation
min_correlation_stocks = correlation_matrix.min().nsmallest(30).index

# Subset the original DataFrame with the selected stocks
selected_stocks = df[min_correlation_stocks]

expected_returns = df.mean()

weights = cp.Variable(len(df.columns))

expected_returns = df.mean()
covariance_matrix = df.cov()

In [16]:
# Calculate the daily returns for each stock
returns_df = df.pct_change().dropna()

# Calculate the expected returns and covariance matrix
expected_returns = returns_df.mean()
covariance_matrix = returns_df.cov()

# Define the objective function to maximize the Sharpe ratio
def objective_function(weights):
    returns = np.dot(weights, expected_returns)
    volatility = np.sqrt(np.dot(weights, np.dot(covariance_matrix, weights.T)))
    sharpe_ratio = returns / volatility
    return -sharpe_ratio  # Negate for maximization

# Define the constraints (e.g., weights sum up to 1)
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights equal to 1
               {'type': 'ineq', 'fun': lambda x: x}]  # Non-negative weights

# Define the bounds for the weights (0 <= weight <= 1)
bounds = [(0, 1)] * len(df.columns)

# Set an initial guess for the weights
initial_guess = [1 / len(df.columns)] * len(df.columns)

# Solve the optimization problem
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = result.x

# Create a DataFrame with the optimal portfolio weights
portfolio_df = pd.DataFrame({'Stock': df.columns, 'Weight': optimal_weights})

# Sort the DataFrame by weights in descending order
portfolio_df = portfolio_df.sort_values('Weight', ascending=False)

In [17]:
# Normalize the weights of the top 30 stocks so that they sum up to 1
selected_stocks = portfolio_df[:30]
normalized_weights = selected_stocks['Weight'] / np.sum(selected_stocks['Weight'])

# Assign the normalized weights back to the selected_weights DataFrame
selected_stocks['Normalized_Weight'] = normalized_weights

selected_stocks = selected_stocks.drop(columns=['Weight'])

selected_stocks 

/var/folders/l_/s2stp_4940d5h7pgt3_4rl040000gn/T/ipykernel_4023/995650356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_stocks['Normalized_Weight'] = normalized_weights


,Stock,Normalized_Weight
258,NEE_close,1.962632e-01
50,BA_close,1.285646e-01
30,AMT_close,1.017679e-01
265,NRG_close,8.896316e-02
372,VRSN_close,7.973063e-02
78,CME_close,7.806501e-02
319,SBAC_close,6.808258e-02
367,VFC_close,5.771036e-02
122,EL_close,4.532964e-02
286,PGR_close,4.532098e-02


In [18]:
selected_stocks.to_csv('selected_stocks(historical)2.csv',index=False)